### **Importing Necessary Libraries**

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### **Importing MYSQL Python Library to get data**

In [38]:
import mysql.connector

#### **Creating the connector to the database**

In [39]:
# Database connection details
config = {'user':'root',
          'host':'localhost',
          'password':'********',
          'database':'example'
}

# Create a Connection to the database
conn = mysql.connector.connect(**config)

# Defining Queries
query_users = "SELECT * FROM users;"
query_logins = "SELECT * FROM logins;"

# Fetch the data into pandas data frame
df_users = pd.read_sql(query_users,conn)
df_logins = pd.read_sql(query_logins,conn)

# Close the database connection
conn.close()

C:\Users\DELL\AppData\Local\Temp\ipykernel_9260\174692745.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_users = pd.read_sql(query_users,conn)
C:\Users\DELL\AppData\Local\Temp\ipykernel_9260\174692745.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_logins = pd.read_sql(query_logins,conn)


In [40]:
df_users.head(5)

,USER_ID,USER_NAME,USER_STATUS
0,1,Alice,Active
1,2,Bob,Inactive
2,3,Charlie,Active
3,4,David,Active
4,5,Eve,Inactive


In [41]:
df_logins.head(5)

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE
0,1,2023-07-15 09:30:00,1001,85
1,2,2023-07-22 10:00:00,1002,90
2,3,2023-08-10 11:15:00,1003,75
3,4,2023-08-20 14:00:00,1004,88
4,5,2023-09-05 16:45:00,1005,82


### **Q1.)Management wants to see all the users that did not login in the past 5 months.Return Username**

In [42]:
df_logins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   USER_ID          28 non-null     int64         
 1   LOGIN_TIMESTAMP  28 non-null     datetime64[ns]
 2   SESSION_ID       28 non-null     int64         
 3   SESSION_SCORE    28 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.0 KB


In [43]:
# caculating the last_login_date
df_logins.head(4)

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE
0,1,2023-07-15 09:30:00,1001,85
1,2,2023-07-22 10:00:00,1002,90
2,3,2023-08-10 11:15:00,1003,75
3,4,2023-08-20 14:00:00,1004,88


In [44]:
last_login_df = df_logins.groupby('USER_ID')['LOGIN_TIMESTAMP'].max().reset_index()

In [45]:
last_login_df['LAST_LOGIN'] = last_login_df['LOGIN_TIMESTAMP'].dt.date
last_login_df

,USER_ID,LOGIN_TIMESTAMP,LAST_LOGIN
0,1,2024-01-10 07:45:00,2024-01-10
1,2,2024-01-25 09:30:00,2024-01-25
2,3,2024-02-05 11:00:00,2024-02-05
3,4,2024-03-01 14:30:00,2024-03-01
4,5,2024-03-15 16:00:00,2024-03-15
5,6,2024-04-12 08:00:00,2024-04-12
6,7,2024-05-18 09:15:00,2024-05-18
7,8,2024-05-28 10:45:00,2024-05-28
8,9,2024-06-15 13:30:00,2024-06-15
9,10,2024-06-28 15:45:00,2024-06-28


In [134]:
from datetime import timedelta,datetime
# Calculate the cutoff date (5 months ago from the current date)
# date_string = '2024-06-28'
# cutoff_date =  datetime.strptime(date_string,'%Y-%m-%d')- timedelta(days=5*30)
current_date = datetime.now()
cutoff_date = current_date - timedelta(days=5*30)
cutoff_date = cutoff_date.date()

In [135]:
cutoff_date

datetime.date(2024, 2, 25)

In [48]:
# Filter users who have not logged in in the last 5 months
result_df = last_login_df[last_login_df['LAST_LOGIN'] < cutoff_date]
result_df

,USER_ID,LOGIN_TIMESTAMP,LAST_LOGIN
0,1,2024-01-10 07:45:00,2024-01-10
1,2,2024-01-25 09:30:00,2024-01-25
2,3,2024-02-05 11:00:00,2024-02-05


### Q2.) For the business units quaterly analysis,calculate how many users and how many sessions were at each quarter order by quarter from newest to oldest.return :first day of the quarter,user_cnt,session_cnt

In [49]:
df_logins.head(4)

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE
0,1,2023-07-15 09:30:00,1001,85
1,2,2023-07-22 10:00:00,1002,90
2,3,2023-08-10 11:15:00,1003,75
3,4,2023-08-20 14:00:00,1004,88


In [50]:
df_quarters = df_logins.copy()

In [51]:
# Finding the quarters in a month
df_quarters['QUARTER'] = df_quarters['LOGIN_TIMESTAMP'].dt.quarter
df_quarters.head()

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE,QUARTER
0,1,2023-07-15 09:30:00,1001,85,3
1,2,2023-07-22 10:00:00,1002,90,3
2,3,2023-08-10 11:15:00,1003,75,3
3,4,2023-08-20 14:00:00,1004,88,3
4,5,2023-09-05 16:45:00,1005,82,3


In [52]:
df_quarters['YEAR'] = (df_quarters['LOGIN_TIMESTAMP'].dt.year) 
df_quarters

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE,QUARTER,YEAR
0,1,2023-07-15 09:30:00,1001,85,3,2023
1,2,2023-07-22 10:00:00,1002,90,3,2023
2,3,2023-08-10 11:15:00,1003,75,3,2023
3,4,2023-08-20 14:00:00,1004,88,3,2023
4,5,2023-09-05 16:45:00,1005,82,3,2023
5,6,2023-10-12 08:30:00,1006,77,4,2023
6,7,2023-11-18 09:00:00,1007,81,4,2023
7,8,2023-12-01 10:30:00,1008,84,4,2023
8,9,2023-12-15 13:15:00,1009,79,4,2023
9,10,2024-06-25 15:00:00,1010,92,2,2024


In [53]:
def get_first_day_of_quarter(year,quarter):
    month = quarter * 3 - 2
    return pd.Timestamp(year,month,1)

df_quarters['FIRST_DAY_OF_QUARTER'] = df_quarters.apply(lambda row:get_first_day_of_quarter(row['YEAR'],row['QUARTER']),axis=1)                            

In [54]:
df_quarters

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE,QUARTER,YEAR,FIRST_DAY_OF_QUARTER
0,1,2023-07-15 09:30:00,1001,85,3,2023,2023-07-01
1,2,2023-07-22 10:00:00,1002,90,3,2023,2023-07-01
2,3,2023-08-10 11:15:00,1003,75,3,2023,2023-07-01
3,4,2023-08-20 14:00:00,1004,88,3,2023,2023-07-01
4,5,2023-09-05 16:45:00,1005,82,3,2023,2023-07-01
5,6,2023-10-12 08:30:00,1006,77,4,2023,2023-10-01
6,7,2023-11-18 09:00:00,1007,81,4,2023,2023-10-01
7,8,2023-12-01 10:30:00,1008,84,4,2023,2023-10-01
8,9,2023-12-15 13:15:00,1009,79,4,2023,2023-10-01
9,10,2024-06-25 15:00:00,1010,92,2,2024,2024-04-01


In [55]:
#Groupby quarter calculate counts
df_groupby = df_quarters.groupby(['FIRST_DAY_OF_QUARTER','QUARTER']).agg(user_count=('USER_ID','nunique'),session_count=('SESSION_ID','count')).reset_index()
df_groupby

,FIRST_DAY_OF_QUARTER,QUARTER,user_count,session_count
0,2023-07-01,3,5,5
1,2023-10-01,4,6,7
2,2024-01-01,1,5,8
3,2024-04-01,2,5,8


### Q3.) Display user id's that login in january 2024 and did not login on november 2023.Return id

In [56]:
df_logins.head()

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE
0,1,2023-07-15 09:30:00,1001,85
1,2,2023-07-22 10:00:00,1002,90
2,3,2023-08-10 11:15:00,1003,75
3,4,2023-08-20 14:00:00,1004,88
4,5,2023-09-05 16:45:00,1005,82


In [57]:
df_logins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   USER_ID          28 non-null     int64         
 1   LOGIN_TIMESTAMP  28 non-null     datetime64[ns]
 2   SESSION_ID       28 non-null     int64         
 3   SESSION_SCORE    28 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.0 KB


In [58]:
# Filter logins for january 2024
january_2024_logins = df_logins[(df_logins['LOGIN_TIMESTAMP'].dt.year == 2024) & (df_logins['LOGIN_TIMESTAMP'].dt.month == 1)]
# Filter logins for November 2023
november_2023_logins = df_logins[(df_logins['LOGIN_TIMESTAMP'].dt.year == 2023) & (df_logins['LOGIN_TIMESTAMP'].dt.month == 11)]

In [59]:
january_2024_logins

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE
10,1,2024-01-10 07:45:00,1011,86
11,2,2024-01-25 09:30:00,1012,89
22,1,2024-01-10 07:45:00,1101,86
23,3,2024-01-25 09:30:00,1102,89
24,5,2024-01-15 11:00:00,1103,78


In [60]:
november_2023_logins

,USER_ID,LOGIN_TIMESTAMP,SESSION_ID,SESSION_SCORE
6,7,2023-11-18 09:00:00,1007,81
25,2,2023-11-10 07:45:00,1201,82
26,4,2023-11-25 09:30:00,1202,84
27,6,2023-11-15 11:00:00,1203,80


In [61]:
#Get user ids
january_2024_user_ids = set(january_2024_logins['USER_ID'])
november_2023_user_ids = set(november_2023_logins['USER_ID'])
january_2024_user_ids,november_2023_user_ids

({1, 2, 3, 5}, {2, 4, 6, 7})

In [62]:
# Finding users who logged in january 2024 but not in november 2024
result_user_ids = january_2024_user_ids - november_2023_user_ids
result_user_ids

{1, 3, 5}

In [63]:
# Converting the result to DataFrame
result_df = pd.DataFrame(result_user_ids,columns=['USER_ID'])
result_df

,USER_ID
0,1
1,3
2,5


### Q4.)Add to the Query 2 the percentage change in sessions from the last quarter.Return: first day of the quarter,session_cnt,session_cnt_prev,session_percent_change.

In [64]:
df_groupby

,FIRST_DAY_OF_QUARTER,QUARTER,user_count,session_count
0,2023-07-01,3,5,5
1,2023-10-01,4,6,7
2,2024-01-01,1,5,8
3,2024-04-01,2,5,8


In [65]:
df_groupby['PRE_SESSION_COUNT'] = df_groupby['session_count'].shift(1)
df_groupby

,FIRST_DAY_OF_QUARTER,QUARTER,user_count,session_count,PRE_SESSION_COUNT
0,2023-07-01,3,5,5,NaN
1,2023-10-01,4,6,7,5.0
2,2024-01-01,1,5,8,7.0
3,2024-04-01,2,5,8,8.0


In [66]:
# df_groupby['PRE_SESSION_COUNT'] = df_groupby['PRE_SESSION_COUNT'].fillna(0).astype('int')
# df_groupby

In [68]:
df_groupby['session_percent_change'] = round((df_groupby['session_count'] - df_groupby['PRE_SESSION_COUNT'])*100/df_groupby['PRE_SESSION_COUNT'],2)
df_groupby

,FIRST_DAY_OF_QUARTER,QUARTER,user_count,session_count,PRE_SESSION_COUNT,session_percent_change
0,2023-07-01,3,5,5,NaN,NaN
1,2023-10-01,4,6,7,5.0,40.00
2,2024-01-01,1,5,8,7.0,14.29
3,2024-04-01,2,5,8,8.0,0.00


### Q5.)Display the user that had the highest session score (max) for each day.Return Date,Username,Score

In [70]:
df_highest = df_logins.copy()

In [71]:
df_highest['LOGIN_DATE'] = df_highest['LOGIN_TIMESTAMP'].dt.date

In [74]:
df_highest.drop('LOGIN_TIMESTAMP',inplace=True,axis=1)

In [75]:
df_highest

,USER_ID,SESSION_ID,SESSION_SCORE,LOGIN_DATE
0,1,1001,85,2023-07-15
1,2,1002,90,2023-07-22
2,3,1003,75,2023-08-10
3,4,1004,88,2023-08-20
4,5,1005,82,2023-09-05
5,6,1006,77,2023-10-12
6,7,1007,81,2023-11-18
7,8,1008,84,2023-12-01
8,9,1009,79,2023-12-15
9,10,1010,92,2024-06-25


In [94]:
agg_df = df_highest.groupby(['LOGIN_DATE','USER_ID'],as_index=False)['SESSION_SCORE'].sum()

In [95]:
agg_df.head()

,LOGIN_DATE,USER_ID,SESSION_SCORE
0,2023-07-15,1,85
1,2023-07-22,2,90
2,2023-08-10,3,75
3,2023-08-20,4,88
4,2023-09-05,5,82


In [110]:
agg_df['rn'] = agg_df.groupby('LOGIN_DATE')['SESSION_SCORE'].rank(method='first',ascending=False).astype(int)
agg_df

,LOGIN_DATE,USER_ID,SESSION_SCORE,rn
0,2023-07-15,1,85,1
1,2023-07-22,2,90,1
2,2023-08-10,3,75,1
3,2023-08-20,4,88,1
4,2023-09-05,5,82,1
5,2023-10-12,6,77,1
6,2023-11-10,2,82,1
7,2023-11-15,6,80,1
8,2023-11-18,7,81,1
9,2023-11-25,4,84,1


In [111]:
result_df = agg_df[agg_df['rn'] == 1]
result_df

,LOGIN_DATE,USER_ID,SESSION_SCORE,rn
0,2023-07-15,1,85,1
1,2023-07-22,2,90,1
2,2023-08-10,3,75,1
3,2023-08-20,4,88,1
4,2023-09-05,5,82,1
5,2023-10-12,6,77,1
6,2023-11-10,2,82,1
7,2023-11-15,6,80,1
8,2023-11-18,7,81,1
9,2023-11-25,4,84,1


### -- Q6.) To identify our best users - Return the users that had a session on every single day .since their firt login (make assumptions if needed).Return: User_id 

In [121]:
df = df_logins.copy()
df['login_date'] = df['LOGIN_TIMESTAMP'].dt.date

# Compute the fixed date for comparison
fixed_date = pd.to_datetime('2024-06-28')

# Group by USER_ID and aggregate
agg_df = df.groupby('USER_ID').agg(
    first_login=('login_date', 'min'),
    no_of_days_login=('login_date', 'nunique')
).reset_index()

In [122]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   USER_ID           10 non-null     int64 
 1   first_login       10 non-null     object
 2   no_of_days_login  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes


In [123]:
agg_df['first_login'] = pd.to_datetime(agg_df['first_login'])
agg_df

,USER_ID,first_login,no_of_days_login
0,1,2023-07-15,2
1,2,2023-07-22,3
2,3,2023-08-10,3
3,4,2023-08-20,3
4,5,2023-09-05,3
5,6,2023-10-12,3
6,7,2023-11-18,2
7,8,2023-12-01,2
8,9,2023-12-15,2
9,10,2024-06-25,4


In [124]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   USER_ID           10 non-null     int64         
 1   first_login       10 non-null     datetime64[ns]
 2   no_of_days_login  10 non-null     int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 372.0 bytes


In [125]:
# Calculate the number of days required
agg_df['no_of_logins_days_required'] = (fixed_date - agg_df['first_login']).dt.days + 1

In [126]:
# Filter: Ensure that number of days required equals the number of days logged in
result_df = agg_df[
    agg_df['no_of_logins_days_required'] == agg_df['no_of_days_login']
]

# Order by USER_ID
result_df = result_df.sort_values(by='USER_ID')

print(result_df)

   USER_ID first_login  no_of_days_login  no_of_logins_days_required
9       10  2024-06-25                 4                           4


### Q7.) On what dates there were no logins at all ? Return : Login_Dates

In [136]:
df = df_logins.copy()
df['login_date'] = df['LOGIN_TIMESTAMP'].dt.date

# Identify the range of dates in the dataset
start_date = df['login_date'].min()
end_date = df['login_date'].max()

# Generate a complete list of dates within the range
all_dates = pd.date_range(start=start_date, end=end_date).date

# Find dates with no logins
logged_dates = df['login_date'].unique()
missing_dates = [date for date in all_dates if date not in logged_dates]

# Create a DataFrame for the missing dates
missing_dates_df = pd.DataFrame(missing_dates, columns=['Login_Date'])

In [137]:
missing_dates_df.head()

,Login_Date
0,2023-07-16
1,2023-07-17
2,2023-07-18
3,2023-07-19
4,2023-07-20
